In [1]:
from EPInformer.models import EPInformer_v2, enhancer_predictor_256bp
from scripts.utils import prepare_input
import scripts.utils_forTraining as train
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
import torch
from torch.utils.data import Subset, Dataset
%load_ext autoreload
%autoreload 2

In [2]:
print(torch.version.cuda)

12.6


## Reproduce prediction results

In [2]:
# 12-fold cross-validation split
split_df = pd.read_csv('./data/leave_chrom_out_crossvalidation_split_18377genes.csv', index_col=0)

In [3]:
def filter_id_lists(existing_ids, train_ids, valid_ids, test_ids):
    """
    Filter the existing IDs to only include those exist in the data
    """
    filtered_train_ids = [i for i in train_ids if i in existing_ids]
    filtered_valid_ids = [i for i in valid_ids if i in existing_ids]
    filtered_test_ids = [i for i in test_ids if i in existing_ids]
    
    return filtered_train_ids, filtered_valid_ids, filtered_test_ids

In [ ]:
cell = 'K562'
distance_threshold = 100_000
n_enhancers = 60
device = 'cuda'
# num_feature == 1: distance; num_feature == 2: distance + enhancer activity; num_feature == 3: distance + enhancer activity + hic contacts
n_extraFeat = 1
batch_size = 16
expr_type = 'RNA'
prediction_res = []
for fi in range(1, 13):
    print("-"*10, 'fold', fi, '-'*10)
    fold_i = 'fold_' + str(fi)

    train_ensid = split_df[split_df[fold_i] == 'train'].index
    valid_ensid = split_df[split_df[fold_i] == 'valid'].index
    test_ensid = split_df[split_df[fold_i] == 'test'].index

    all_ds = train.promoter_enhancer_dataset(expr_type=expr_type, cell_type=cell, n_extraFeat=n_extraFeat, usePromoterSignal=True, 
                                             n_enhancers=n_enhancers, distance_threshold=distance_threshold, data_folder = './data/', rna_seq_source='epiatlas')


    ensid_list = [eid.decode() for eid in all_ds.data_h5['ensid'][:]]
    existing_ensids = all_ds.get_valid_genes()
    ensid_df = pd.DataFrame(ensid_list, columns=['ensid'])
    ensid_df['idx'] = np.arange(len(ensid_list))
    ensid_df = ensid_df.set_index('ensid')
    train_ensid, valid_ensid, test_ensid = filter_id_lists(existing_ensids, train_ensid, valid_ensid, test_ensid)
    train_idx = ensid_df.loc[train_ensid]['idx']
    valid_idx = ensid_df.loc[valid_ensid]['idx']
    test_idx = ensid_df.loc[test_ensid]['idx']

    train_ds = Subset(all_ds, train_idx)
    valid_ds = Subset(all_ds, valid_idx)
    test_ds = Subset(all_ds, test_idx)

    model = EPInformer_v2(n_encoder=3, n_enhancer=n_enhancers, out_dim=64, n_extraFeat=n_extraFeat, device=device)
    model = model.to(device)
    # model_name= 'tunedEPInformerV2.preTrainedConv.4base.64dim.3Trans.4head.TrueBN.TrueLN.TrueFeat.3extraFeat.60enh.K562.rmEnhNone.bs16.seq_feat_dist.DNaseH.distanceDist100k.hic0.len2k.distance.{}'.format(expr_type)
    # checkpoint = torch.load("./trained_models/EPInformer_PE_Activity_HiC/K562/fold_{}_EPInformer_PE_Activity_HiC_{}_K562_checkpoint.pt".format(fi, expr_type), weights_only=False)
    checkpoint = torch.load(f"./trained_models/2025-05-09-14/fold_{fi}_best_EPInformer-PE.preTrainedConv.4base.64dim.3Trans.4head.TrueBN.TrueLN.TrueFeat.1extraFeat.60enh.K562.RNA_checkpoint.pt", weights_only=False)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    test_df = train.test(model, test_ds, fold_i=fi, batch_size=batch_size, device=device)
    prediction_res.append(test_df)
prediction_res = pd.concat(prediction_res)

---------- fold 1 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [00:27<00:00,  5.01it/s]


PearsonR: 0.73964125
---------- fold 2 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83/83 [00:28<00:00,  2.95it/s]


PearsonR: 0.6963873
---------- fold 3 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [00:19<00:00,  4.11it/s]


PearsonR: 0.75337845
---------- fold 4 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:24<00:00,  3.49it/s]


PearsonR: 0.7322423
---------- fold 5 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:23<00:00,  4.64it/s]


PearsonR: 0.7100311
---------- fold 6 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118/118 [00:21<00:00,  5.41it/s]


PearsonR: 0.7058993
---------- fold 7 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:14<00:00,  3.81it/s]


PearsonR: 0.7817382
---------- fold 8 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:24<00:00,  5.22it/s]


PearsonR: 0.67451954
---------- fold 9 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:15<00:00,  5.05it/s]


PearsonR: 0.73240674
---------- fold 10 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:15<00:00,  5.69it/s]


PearsonR: 0.7486443
---------- fold 11 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:13<00:00,  5.29it/s]


PearsonR: 0.6976114
---------- fold 12 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:16<00:00,  7.26it/s]

PearsonR: 0.7254212


In [9]:
pearsonR, pv = stats.pearsonr(prediction_res['Pred'], prediction_res['actual'])
print('PearsonR of 12-fold cross-validation predicitons', pearsonR, 'Total genes:', len(prediction_res))

PearsonR of 12-fold cross-validation predicitons 0.7107365 Total genes: 18360


In [10]:
prediction_res

,Pred,actual,fold_idx
ENSG00000002016,0.318330,1.010724,1
ENSG00000003056,0.997860,1.940716,1
ENSG00000003137,-0.669042,-0.958607,1
ENSG00000003393,0.243691,1.010300,1
ENSG00000003400,-0.788664,1.224274,1
...,...,...,...
ENSG00000255835,0.986564,-1.000000,12
ENSG00000256374,-1.092015,-1.000000,12
ENSG00000257315,0.581756,0.741939,12
ENSG00000258465,0.883309,-0.376751,12


<Figure size 600x600 with 0 Axes>

In [11]:
plt.figure(figsize=(6,6))
ax = sns.jointplot(
    data=prediction_res,
    x="Pred",
    y="actual",
    kind = 'scatter',
    joint_kws={'marker':'o', 's':10, 'alpha':0.1, 'linewidth':0},
    marginal_kws={'bins':20, 'element':'step', 'kde':True, 'linewidth':0},
)
ax.plot_joint(sns.regplot, color="r", scatter=False, line_kws={"color": "orange", 'linestyle':'dashed'})
plt.title('pearsonR = {:.3f}'.format(pearsonR))
plt.ylabel('K562 RNA-seq expression')
plt.xlabel('12-fold cross-chromosome predictions')
plt.tight_layout()

In [7]:
cell = 'GM12878'
distance_threshold = 100_000
n_enhancers = 60
device = 'cuda'
# num_feature == 1: distance; num_feature == 2: distance + enhancer activity; num_feature == 3: distance + enhancer activity + hic contacts
n_extraFeat = 1
batch_size = 16
expr_type = 'RNA'
prediction_res = []
for fi in range(1, 13):
    print("-"*10, 'fold', fi, '-'*10)
    fold_i = 'fold_' + str(fi)

    train_ensid = split_df[split_df[fold_i] == 'train'].index
    valid_ensid = split_df[split_df[fold_i] == 'valid'].index
    test_ensid = split_df[split_df[fold_i] == 'test'].index

    all_ds = train.promoter_enhancer_dataset(data_folder='data', expr_type=expr_type, cell_type=cell, n_extraFeat=n_extraFeat, 
                                             n_enhancers=n_enhancers, distance_threshold=distance_threshold, rna_seq_source='epiatlas')
    existing_ensids = all_ds.get_valid_genes()
    ensid_list = [eid.decode() for eid in all_ds.data_h5['ensid'][:]]
    ensid_df = pd.DataFrame(ensid_list, columns=['ensid'])
    ensid_df['idx'] = np.arange(len(ensid_list))
    ensid_df = ensid_df.set_index('ensid')
    train_ensid, valid_ensid, test_ensid = filter_id_lists(existing_ensids, train_ensid, valid_ensid, test_ensid)
    train_idx = ensid_df.loc[train_ensid]['idx']
    valid_idx = ensid_df.loc[valid_ensid]['idx']
    test_idx = ensid_df.loc[test_ensid]['idx']

    train_ds = Subset(all_ds, train_idx)
    valid_ds = Subset(all_ds, valid_idx)
    test_ds = Subset(all_ds, test_idx)

    model = EPInformer_v2(n_encoder=3, n_enhancer=n_enhancers, out_dim=64, n_extraFeat=n_extraFeat, device=device)
    checkpoint = torch.load(f"./trained_models/2025-05-08-12/fold_{fi}_best_EPInformer-PE.preTrainedConv.4base.64dim.3Trans.4head.TrueBN.TrueLN." \
                            "TrueFeat.1extraFeat.60enh.GM12878.RNA_checkpoint.pt", weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    test_df = train.test(model, test_ds, fold_i=fi, batch_size=batch_size, device=device)
    prediction_res.append(test_df)
prediction_res = pd.concat(prediction_res)

---------- fold 1 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [00:24<00:00,  5.63it/s]


PearsonR: 0.7618537
---------- fold 2 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83/83 [00:10<00:00,  8.06it/s]


PearsonR: 0.7848559
---------- fold 3 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [00:12<00:00,  6.45it/s]


PearsonR: 0.7437443
---------- fold 4 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:22<00:00,  3.88it/s]


PearsonR: 0.75515676
---------- fold 5 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:18<00:00,  5.83it/s]


PearsonR: 0.6915893
---------- fold 6 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118/118 [00:25<00:00,  4.57it/s]


PearsonR: 0.7001714
---------- fold 7 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:10<00:00,  5.58it/s]


PearsonR: 0.7424082
---------- fold 8 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:30<00:00,  4.26it/s]


PearsonR: 0.7488134
---------- fold 9 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:19<00:00,  3.98it/s]


PearsonR: 0.7670057
---------- fold 10 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:16<00:00,  5.24it/s]


PearsonR: 0.7667726
---------- fold 11 ----------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:12<00:00,  5.94it/s]


PearsonR: 0.741115
---------- fold 12 ----------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:19<00:00,  6.24it/s]

PearsonR: 0.7469388


In [8]:
pearsonR, pv = stats.pearsonr(prediction_res['Pred'], prediction_res['actual'])
print('PearsonR of 12-fold cross-validation predicitons', pearsonR, 'Total genes:', len(prediction_res))

PearsonR of 12-fold cross-validation predicitons 0.7361207 Total genes: 18360


<Figure size 600x600 with 0 Axes>

In [9]:
plt.figure(figsize=(6,6))
ax = sns.jointplot(
    data=prediction_res,
    x="Pred",
    y="actual",
    kind = 'scatter',
    joint_kws={'marker':'o', 's':10, 'alpha':0.1, 'linewidth':0},
    marginal_kws={'bins':20, 'element':'step', 'kde':True, 'linewidth':0},
)
ax.plot_joint(sns.regplot, color="r", scatter=False, line_kws={"color": "orange", 'linestyle':'dashed'})
plt.title('pearsonR = {:.3f}'.format(pearsonR))
plt.ylabel('GM12878 RNA-seq expression')
plt.xlabel('12-fold cross-chromosome predictions')
plt.tight_layout()

## Predict gene expression given promoter-enhancer sequence

In [8]:
# Donwload reference genome
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz  -P ./data/
!gunzip ./data/hg38.fa.gz

--2025-05-05 16:57:55--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘./data/hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938,09M  16,8MB/s    in 59s     

2025-05-05 16:58:56 (15,9 MB/s) - ‘./data/hg38.fa.gz’ saved [983659424/983659424]



In [12]:
# Load ABC enhancer-gene data
enhancer_gene_k562 = pd.read_csv('./data/K562_DNase_ENCFF257HEE_hic_4DNFITUOMFUQ_1MB_ABC_nominated/Gene-enhancer links/EnhancerPredictionsAllPutative.txt.gz', sep='\t')
# Select the gene-enhancer links within 100kb to the TSS of target gene and remove the promoter element
enhancer_gene_k562_100kb = enhancer_gene_k562[(enhancer_gene_k562['distance']<=100_000)&(enhancer_gene_k562['distance']>1000)].reset_index()
enhancer_gene_k562_100kb.to_csv('./data/K562_enhancer_gene_links_100kb.tsv', index=False, sep='\t')

In [13]:
enhancer_gene_k562_100kb = pd.read_csv('./data/K562_enhancer_gene_links_100kb.tsv', sep='\t')
gene_tss = pd.read_csv('./data/K562_DNase_ENCFF257HEE_hic_4DNFITUOMFUQ_1MB_ABC_nominated/DNase_ENCFF257HEE_Neighborhoods/GeneList.txt', sep='\t')[['name', 'chr', 'tss', 'strand']]
data_split = pd.read_csv('./data/leave_chrom_out_crossvalidation_split_18377genes.csv')
gene_tss = gene_tss.merge(data_split[['ENSID', 'Gene name']], left_on='name', right_on='ENSID').drop(columns='name')
enhancer_gene_k562_100kb_includeNoEnhancerGene = enhancer_gene_k562_100kb.merge(gene_tss, left_on='TargetGene', right_on='ENSID', how='right', suffixes=['', '_gene']).reset_index()

In [14]:
enhancer_gene_k562_100kb_includeNoEnhancerGene.head(3)

,level_0,index,chr,start,end,name,class,activity_base,TargetGene,TargetGeneTSS,...,ABC.Score.Numerator,ABC.Score,powerlaw.Score.Numerator,powerlaw.Score,CellType,chr_gene,tss,strand,ENSID,Gene name
0,0,355555.0,chrX,100671069.0,100671638.0,genic|chrX:100671069-100671638,genic,0.191760,ENSG00000000003,100636750.0,...,0.485751,0.008531,0.004399,0.018000,K562,chrX,100636750,-,ENSG00000000003,TSPAN6
1,1,355579.0,chrX,100674734.0,100675481.0,genic|chrX:100674734-100675481,genic,0.397338,ENSG00000000003,100636750.0,...,1.176227,0.020658,0.008333,0.034101,K562,chrX,100636750,-,ENSG00000000003,TSPAN6
2,2,355603.0,chrX,100685550.0,100686167.0,genic|chrX:100685550-100686167,genic,0.555460,ENSG00000000003,100636750.0,...,1.509652,0.026514,0.009396,0.038451,K562,chrX,100636750,-,ENSG00000000003,TSPAN6


In [15]:
gene_info = data_split[data_split['fold_1'] == 'test'][['ENSID', 'Gene name']].head(16).reset_index(drop=True)
gene_list = list(gene_info['ENSID'])
# encode gene-enhancer links for EPInformer
# num_feature == 1: distance; num_feature == 2: distance + enhancer activity; num_feature == 3: distance + enhancer activity + hic contacts
device = 'cpu'
PE_codes, PE_feats, mRNA_feats, PE_pairs = prepare_input(enhancer_gene_k562_100kb_includeNoEnhancerGene, gene_list, 'K562', num_features=2)
PE_codes = torch.from_numpy(PE_codes).float().to(device)
PE_feats = torch.from_numpy(PE_feats).float().to(device)
mRNA_feats = torch.from_numpy(mRNA_feats).float().to(device)
print(PE_codes.shape, PE_feats.shape, mRNA_feats.shape)

  0%|                                                                                                                                                           | 0/16 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 11.03it/s]

torch.Size([16, 61, 2000, 4]) torch.Size([16, 61, 2]) torch.Size([16, 9])


In [16]:
# Load pre-trained EPInformer-PE-Activity (CAGE-seq)
expr_type = 'CAGE'
model = EPInformer_v2(n_encoder=3, n_enhancer=60, out_dim=64, n_extraFeat=2, device=device)
model_path = './trained_models/EPInformer_PE_Activity/K562/fold_1_EPInformer_PE_Activity_{}_K562_checkpoint.pt'.format(expr_type)
checkpoint = torch.load(model_path, map_location=torch.device(device), weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

In [17]:
model.eval()
with torch.no_grad():
    pred_expr, _ = model(PE_codes, mRNA_feats, PE_feats)
    pred_expr = pred_expr.numpy().squeeze()
gene_info['predicted_expr'] = pred_expr
gene_info['expr_type'] = expr_type

In [18]:
gene_info

,ENSID,Gene name,predicted_expr,expr_type
0,ENSG00000002016,RAD52,1.913353,CAGE
1,ENSG00000003056,M6PR,2.470468,CAGE
2,ENSG00000003137,CYP26B1,1.000790,CAGE
3,ENSG00000003393,ALS2,1.903482,CAGE
4,ENSG00000003400,CASP10,0.779036,CAGE
5,ENSG00000003402,CFLAR,1.895073,CAGE
6,ENSG00000003436,TFPI,1.848657,CAGE
7,ENSG00000003509,NDUFAF7,2.525692,CAGE
8,ENSG00000004478,FKBP4,1.874264,CAGE
9,ENSG00000004700,RECQL,2.110345,CAGE
